# AutoGen StateFlow for Industrial Control Problem

In this notebook, we demonstrate how to use AutoGen groupchat to build a workflow with various agents from a state-oriented perspective.


## Set API Endpoint

In [6]:
import autogen

config_list = autogen.config_list_from_json(
    "./OAI_CONFIG_LIST.json"  # All needed OpenAI info is in this file
)

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": config_list,
    "temperature": 0,
}


## A workflow for pH low calculation

We define the following agents:
- Initializer: Start the workflow by sending a task.
- Coder: Retrieve papers from the internet by writing code.
- Executor: Execute the code.
- Scientist: Read the papers and write a summary.

In [7]:
import tempfile

from autogen.coding import LocalCommandLineCodeExecutor

temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

initializer = autogen.UserProxyAgent(
    name="Init",
    code_execution_config=False,
    llm_config=llm_config,
)


In [8]:
data_analyst = autogen.AssistantAgent(
    name="data_analyst",
    llm_config=llm_config,
    system_message="""
You are a data analyst who help people find information through mathematic or statistics analyses.
Given a topic, you write code to retrieve the needed information.
You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. 
The user can't modify your code. So do not suggest incomplete code which requires others to modify. 
Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. 
Suggest the full code instead of partial code or code changes. 
If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, 
collect additional info you need, and think of a different approach to try.
""",
)
code_executor = autogen.UserProxyAgent(
    name="code_executor",
    system_message="Executor. Execute the code written by the data analyst and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"executor": executor},
)

scientist = autogen.AssistantAgent(
    name="data scientist",
    llm_config=llm_config,
    system_message="""You are the data scientist. Please summerize the execution results from different parties and provide insights.""",
)



In [9]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is initializer:
        # init -> retrieve
        return data_analyst
    elif last_speaker is data_analyst:
        # retrieve: action 1 -> action 2
        return code_executor
    elif last_speaker is code_executor:
        if messages[-1]["content"] == "exitcode: 1":
            # retrieve --(execution failed)--> retrieve
            return data_analyst
        else:
            # retrieve --(execution sucess)--> research
            return scientist
        return scientist
        # research -> end
        return None


groupchat = autogen.GroupChat(
    agents=[initializer, data_analyst, code_executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [10]:
chat_result = initializer.initiate_chat(
    manager, message='''Given a dataset       
time	pH  
0	2023-01-01 00:00:00	7.0  
1	2023-01-01 01:00:00	6.8  
2	2023-01-01 02:00:00	6.5  
3	2023-01-01 03:00:00	6.2  
4	2023-01-01 04:00:00	6.0  
5	2023-01-01 05:00:00	5.8  
6	2023-01-01 06:00:00	5.5  
7	2023-01-01 07:00:00	5.3  
8	2023-01-01 08:00:00	5.0  
9	2023-01-01 09:00:00	4.8.   
Please calculate the mean_pH, and std_pH. Then calculate the lower band which is defined as lower_band = mean_ph - 2.0 * std.  
Once having the lower_band, please calculate drop_at, the exact time the pH became below the lower_band. 
The value drop_at should be the time when the pH first dropped below the lower_band and initialized as None.
If the pH never drops below the lower_band, make a suggestion of 'Keep monitoring'.
Otherwise, make a suggestion for operator to 'Change Rate'
Always inlcude the values of mean_pH, std_pH, and lower_band in the response.
 '''
)

Init (to chat_manager):

Given a dataset       
time	pH  
0	2023-01-01 00:00:00	7.0  
1	2023-01-01 01:00:00	6.8  
2	2023-01-01 02:00:00	6.5  
3	2023-01-01 03:00:00	6.2  
4	2023-01-01 04:00:00	6.0  
5	2023-01-01 05:00:00	5.8  
6	2023-01-01 06:00:00	5.5  
7	2023-01-01 07:00:00	5.3  
8	2023-01-01 08:00:00	5.0  
9	2023-01-01 09:00:00	4.8.   
Please calculate the mean_pH, and std_pH. Then calculate the lower band which is defined as lower_band = mean_ph - 2.0 * std.  
Once having the lower_band, please calculate drop_at, the exact time the pH became below the lower_band. 
The value drop_at should be the time when the pH first dropped below the lower_band and initialized as None.
If the pH never drops below the lower_band, make a suggestion of 'Keep monitoring'.
Otherwise, make a suggestion for operator to 'Change Rate'
Always inlcude the values of mean_pH, std_pH, and lower_band in the response.
 

--------------------------------------------------------------------------------

Next speake